# Neuronske mreže - Projekt
## Tema: Predviđanje ishoda nogometnih utakmica

**Akademska godina:** 2025./2026.

**Projektni tim**:
- Antun Slaviček - voditelj
- Karin Brajdić
- Karlo Mezdić
- Luka Špiljak
- Nikola Zrnc

------------------------------

## 1. Uvod

### 1.1. Cilj
Cilj projekta je razviti i evaluirati **model neuronske mreže (NN)** sposoban za klasifikaciju ishoda nogometnih utakmica. Model će predviđati jednu od tri klase: **pobjeda domaćina, neriješeno ili pobjeda gosta**, na temelju statističkih i povijesnih značajki timova prije početka susreta.

### 1.2. Korištena Baza Podataka
Projekt koristi [`European Soccer Database`](https://www.kaggle.com/datasets/hugomathien/soccer) s platforme Kaggle (autora Hugo Mathien). Baza sadrži detaljne podatke o više od 25.000 utakmica, 11 europskih liga (od 2008. do 2016. godine), atribute igrača i timova (preuzete iz video igre FIFA) te kladioničarske koeficijente.

### 1.3. Razrada projekta u zadatke
Projekt je podijeljen u sljedeće faze:
1.  **Prikupljanje i analiza podataka:** Učitavanje SQLite baze podataka i razumijevanje odnosa između tablica (`Match`, `Team`, `Player_Attributes`).
2.  **Priprema i obrada podataka:** Spajanje i strukturiranje podataka po utakmicama, timovima i sezonama. Kreiranje novih varijabli (npr. forma tima, prosječna ocjena tima).
3.  **Priprema podataka za NN:** Priprema podataka za NN (skaliranje i podjela na skupove za treniranje, validaciju i testiranje).
4.  **Izgradnja modela:** Implementacija i treniranje neuronske mreže.
5.  **Evaluacija:** Mjerenje uspješnosti modela.

------------------------------

## 2. Prikaz i obrada podataka

### 2.1. Učitavanje biblioteka

In [12]:
import kagglehub
import sqlite3
import pandas as pd
import numpy as np
import os

### 2.2. Dohvat i spajanje na bazu

In [13]:
path = kagglehub.dataset_download("hugomathien/soccer")
database_path = os.path.join(path, "database.sqlite")

conn = sqlite3.connect(database_path)
print(f"Uspješno spojeni na SQLite bazu na putanji: {database_path}")

tables = pd.read_sql("SELECT name FROM sqlite_master WHERE type='table';", conn)
print("\nDostupne tablice:")
print(tables)

Uspješno spojeni na SQLite bazu na putanji: C:\Users\Toni\.cache\kagglehub\datasets\hugomathien\soccer\versions\10\database.sqlite

Dostupne tablice:
                name
0    sqlite_sequence
1  Player_Attributes
2             Player
3              Match
4             League
5            Country
6               Team
7    Team_Attributes


### 2.3. Učitavanje podataka

Kako bismo predvidjeli ishod utakmica, moramo pravilno izabrati podatke koje ćemo učitati te obrađivati.

Tako su nam najvažnije tablice:
- **Match**: sadrži sve odigrane utakmice, njihove rezultate te igrače koji su igrali na utakmici
- **Player_Attributes**: sarži informacije o kvaliteti i ocjenama igrača
- **Team_Attributes**: sadrži informacije o timovima i njihovoj kvaliteti igre

In [14]:
df_match = pd.read_sql("SELECT * FROM Match", conn)

df_team = pd.read_sql("SELECT * FROM Team", conn)
df_team_att = pd.read_sql("SELECT * FROM Team_Attributes", conn)

df_player = pd.read_sql("SELECT * FROM Player", conn)
df_player_att = pd.read_sql("SELECT * FROM Player_Attributes", conn)

df_league = pd.read_sql("SELECT * FROM League", conn)
df_country = pd.read_sql("SELECT * FROM Country", conn)

print("Podaci uspješno učitani!")
print(f"Broj utakmica: {df_match.shape[0]}")
print(f"Broj timova: {df_team.shape[0]}")
print(f"Broj igrača: {df_player.shape[0]}")

Podaci uspješno učitani!
Broj utakmica: 25979
Broj timova: 299
Broj igrača: 11060


### 2.3.1. Ekstrakcija podataka o utakmicama (Tablica `Match`)

Tablica `Match` predstavlja osnovu našeg skupa podataka. Iz nje izdvajamo:
* **Identifikatore i vrijeme:** `match_api_id`, `home_team_api_id`, `away_team_api_id`, `date`.
* **Sastave timova:** ID-jevi 11 igrača za domaću i gostujuću ekipu (ključno za povezivanje s ocjenama igrača).
* **Kladioničarske koeficijente:** (npr. B365H, B365D, B365A) koji služe kao ekspertska procjena vjerojatnosti ishoda.
* **Rezultat:** Broj postignutih golova.

In [15]:
match_stupci = ['season', 'date', 'home_team_api_id', 'away_team_api_id', 'home_team_goal', 'away_team_goal']

display(df_match[match_stupci].head())

,season,date,home_team_api_id,away_team_api_id,home_team_goal,away_team_goal
0,2008/2009,2008-08-17 00:00:00,9987,9993,1,1
1,2008/2009,2008-08-16 00:00:00,10000,9994,0,0
2,2008/2009,2008-08-16 00:00:00,9984,8635,0,3
3,2008/2009,2008-08-17 00:00:00,9991,9998,5,0
4,2008/2009,2008-08-16 00:00:00,7947,9985,1,3


### 2.3.2. Metrika kvalitete igrača (Tablica `Player_Attributes`)

Ova tablica sadrži povijesne ocjene igrača preuzete iz FIFA videoigre. Ključna varijabla je `overall_rating` koja predstavlja kvalitetu igrača u određenom trenutku. Budući da se ocjene mijenjaju kroz vrijeme, ključno je za svaku utakmicu pronaći onu ocjenu koja je bila **aktivna na datum odigravanja susreta**.

In [16]:
player_att_stupci = ['id', 'date', 'overall_rating']

display(df_player_att[player_att_stupci].head())

,id,date,overall_rating
0,1,2016-02-18 00:00:00,67.0
1,2,2015-11-19 00:00:00,67.0
2,3,2015-09-21 00:00:00,62.0
3,4,2015-03-20 00:00:00,61.0
4,5,2007-02-22 00:00:00,61.0


### 2.3.3. Taktika momčadi (Tablica `Team_Attributes`)

Ova tablica definira stil igre momčadi.

In [17]:
team_att_stupci = ['team_api_id', 'date', 'buildUpPlaySpeed', 'buildUpPlayPassing', 'defencePressure']

display(df_team_att[team_att_stupci].head())

,team_api_id,date,buildUpPlaySpeed,buildUpPlayPassing,defencePressure
0,9930,2010-02-22 00:00:00,60,50,50
1,9930,2014-09-19 00:00:00,52,56,47
2,9930,2015-09-10 00:00:00,47,54,47
3,8485,2010-02-22 00:00:00,70,70,60
4,8485,2011-02-22 00:00:00,47,52,47


### 2.4. Konstrukcija finalnog skupa podataka

U ovoj fazi spajamo podatke iz relacijskih tablica u jedinstveni `DataFrame`. Proces uključuje sljedeće ključne korake:

1.  **Diskretizacija ciljne varijable (`result`):**
    Pretvorba numeričkog rezultata (golova) u kategorijsku varijablu za klasifikaciju:
    * **1:** Pobjeda domaćina (Home Win)
    * **0:** Neriješeno (Draw)
    * **2:** Pobjeda gosta (Away Win)

2.  **Vremenska sinkronizacija kvalitete igrača:**
    Iteriramo kroz svakog od 22 igrača na terenu te pronalazimo njegov `overall_rating` na dan utakmice. Kako bismo smanjili dimenzionalnost, ne koristimo 22 zasebne značajke, već izračunavamo **prosječni rating** za domaću (`home_team_rating`) i gostujuću (`away_team_rating`) postavu.

3.  **Pridruživanje taktičkih atributa:**
    Slično kao i za igrače, za svaki tim pronalazimo atribute (`buildUpPlaySpeed`, `defencePressure`, itd.) koji su bili aktualni u trenutku utakmice, dajući modelu uvid u stil igre.

4.  **Izračun forme timova:**
    Jedan od najvažnijih prediktora je trenutna forma. Implementirali smo logiku koja za svaku utakmicu gleda **posljednjih 5 susreta** i računa:
    * `team_form_points`: Prosječan broj osvojenih bodova u zadnjih 5 kola.
    * `avg_goals_scored`: Prosječna efikasnost napada.
    * `avg_goals_conceded`: Prosječna propusnost obrane.

5.  **Finalna obrada i imputacija:**
    Uklanjanje nepotrebnih identifikatora te popunjavanje nedostajućih vrijednosti (`NaN`) srednjim vrijednostima kako bi se sačuvao integritet skupa podataka.

### 2.4.1. Rezultat

Model neuronske mreže zahtijeva numeričke oznake za klase.
Pretvaramo gol-razliku u kategoriju `result`:
* Ako `home_goal > away_goal` -> Klasa **1**
* Ako `home_goal == away_goal` -> Klasa **0**
* Ako `home_goal < away_goal` -> Klasa **2**

In [18]:
def vrati_rezultat(redak):
    if redak['home_team_goal'] > redak['away_team_goal']:
        return 1
    elif redak['home_team_goal'] == redak['away_team_goal']:
        return 0
    else:
        return 2

df_match['result'] = df_match.apply(vrati_rezultat, axis=1)

### 2.4.2.  Prosječna ocjena

Izazov u ovom koraku je što se ocjene igrača (iz FIFA baze) ažuriraju više puta godišnje. Za svaku utakmicu moramo pronaći **posljednju dostupnu ocjenu igrača prije datuma utakmice**.

Algoritam:
1.  Grupiramo atribute po igračima i sortiramo ih kronološki.
2.  Za svakog od 22 igrača na terenu tražimo ocjenu (`overall_rating`) koja je bila aktualna na dan utakmice.
3.  Računamo prosjek (`mean`) za domaću i gostujuću ekipu kako bismo dobili varijable `home_team_rating` i `away_team_rating`.

In [19]:
df_match['date'] = pd.to_datetime(df_match['date'])
df_player_att['date'] = pd.to_datetime(df_player_att['date'])

player_ratings = {}
sorted_attributes = df_player_att.sort_values('date')

for id, group in sorted_attributes.groupby('player_api_id'):
    player_ratings[id] = {
        'dates': group['date'].values,
        'ratings': group['overall_rating'].values
    }

def get_rating_for_player(player_id, match_date):
    if pd.isna(player_id) or player_id not in player_ratings:
        return np.nan
    
    data = player_ratings[player_id]
    valid_dates = data['dates'] <= match_date
    
    if not np.any(valid_dates):
        return np.nan
    
    return data['ratings'][valid_dates][-1]

def get_match_ratings(redak):
    home_players = [redak[f'home_player_{i}'] for i in range(1, 12)]
    away_players = [redak[f'away_player_{i}'] for i in range(1, 12)]

    home_ratings = [get_rating_for_player(p, redak['date']) for p in home_players]
    away_ratings = [get_rating_for_player(p, redak['date']) for p in away_players]

    avg_home = np.nanmean(home_ratings)
    avg_away = np.nanmean(away_ratings)

    return pd.Series([avg_home, avg_away])

ratings = df_match.apply(get_match_ratings, axis=1)
ratings.columns = ['home_team_rating', 'away_team_rating']

df_match = pd.concat([df_match, ratings], axis=1)

C:\Users\Toni\AppData\Local\Temp\ipykernel_29460\2980615144.py:32: RuntimeWarning: Mean of empty slice
  avg_home = np.nanmean(home_ratings)
C:\Users\Toni\AppData\Local\Temp\ipykernel_29460\2980615144.py:33: RuntimeWarning: Mean of empty slice
  avg_away = np.nanmean(away_ratings)


### 2.4.3. Spajanje atributa timova

Slično kao kod igrača, karakteristike tima (npr. `defencePressure`, `buildUpPlaySpeed`) mijenjaju se kroz sezone. Funkcija `get_team_attributes` pronalazi stanje karakteristika tima koje je vrijedilo na datum utakmice kako bi model imao uvid u taktički stil igre.

In [9]:
df_team_att['date'] = pd.to_datetime(df_team_att['date'])
df_team_att.sort_values(['team_api_id', 'date'], inplace=True)

def get_team_attributes(match_date, team_id, attributes_df):
    team_stats = attributes_df[
        (attributes_df['team_api_id'] == team_id) & 
        (attributes_df['date'] <= match_date)
    ]
    
    if team_stats.empty:
        return pd.Series(index=['buildUpPlaySpeed', 'defencePressure', 'buildUpPlayPassing'])
    
    return team_stats.iloc[-1][['buildUpPlaySpeed', 'defencePressure', 'buildUpPlayPassing']]

print("Dohvaćanje atributa domaćih timova...")
home_team_stats = df_match.apply(
    lambda x: get_team_attributes(x['date'], x['home_team_api_id'], df_team_att), axis=1
)
home_team_stats.columns = ['home_buildUpSpeed', 'home_defencePressure', 'home_passing']

print("Dohvaćanje atributa gostujućih timova...")
away_team_stats = df_match.apply(
    lambda x: get_team_attributes(x['date'], x['away_team_api_id'], df_team_att), axis=1
)
away_team_stats.columns = ['away_buildUpSpeed', 'away_defencePressure', 'away_passing']

df_match = pd.concat([df_match, home_team_stats, away_team_stats], axis=1)
print("Atributi timova uspješno dodani!")

Dohvaćanje atributa domaćih timova...
Dohvaćanje atributa gostujućih timova...
Atributi timova uspješno dodani!


### 2.4.4. Izračun forme timova

Forma je ključan faktor u sportu.
Za svaku utakmicu gledamo **posljednjih 5 susreta** tog tima i računamo:
* **Bodovni učinak:** Prosjek osvojenih bodova (3 za pobjedu, 1 za remi, 0 za poraz).
* **Napadački učinak:** Prosječan broj postignutih golova.
* **Obrambeni učinak:** Prosječan broj primljenih golova.

*Napomena: Ako tim nema odigranih 5 prethodnih utakmica (početak podataka), vrijednosti se postavljaju na 0.*

In [10]:
team_history = {} 

df_match_sorted = df_match.sort_values('date').copy()

home_form_points = []
away_form_points = []

home_avg_goals_scored = []
away_avg_goals_scored = []
home_avg_goals_conceded = []
away_avg_goals_conceded = []

print("Pokrećem izračun forme (bodovi i golovi) za zadnjih 5 utakmica...")

for index, row in df_match_sorted.iterrows():
    h_id = row['home_team_api_id']
    a_id = row['away_team_api_id']
    
    h_g = row['home_team_goal']
    a_g = row['away_team_goal']

    last_5_home = team_history.get(h_id, [])[-5:]
    
    if last_5_home:
        h_points_sum = sum(x[0] for x in last_5_home)
        home_form_points.append(h_points_sum / len(last_5_home))
        
        h_score_sum = sum(x[1] for x in last_5_home)
        h_concede_sum = sum(x[2] for x in last_5_home)
        
        home_avg_goals_scored.append(h_score_sum / len(last_5_home))
        home_avg_goals_conceded.append(h_concede_sum / len(last_5_home))
        
    else: 
        home_form_points.append(0)
        home_avg_goals_scored.append(0)
        home_avg_goals_conceded.append(0)


    last_5_away = team_history.get(a_id, [])[-5:]

    if last_5_away:
        a_points_sum = sum(x[0] for x in last_5_away)
        away_form_points.append(a_points_sum / len(last_5_away))
        
        a_score_sum = sum(x[1] for x in last_5_away) 
        a_concede_sum = sum(x[2] for x in last_5_away)

        away_avg_goals_scored.append(a_score_sum / len(last_5_away))
        away_avg_goals_conceded.append(a_concede_sum / len(last_5_away))
    else:
        away_form_points.append(0)
        away_avg_goals_scored.append(0)
        away_avg_goals_conceded.append(0)
    

    if row['result'] == 1:
        h_points = 3
        a_points = 0

    elif row['result'] == 0:
        h_points = 1
        a_points = 1

    else:
        h_points = 0
        a_points = 3
        
    if h_id not in team_history: team_history[h_id] = []
    if a_id not in team_history: team_history[a_id] = []
    
    team_history[h_id].append((h_points, h_g, a_g))
    
    team_history[a_id].append((a_points, a_g, h_g))
    
    
df_match_sorted['home_team_form_points'] = home_form_points
df_match_sorted['away_team_form_points'] = away_form_points
df_match_sorted['home_avg_goals_scored'] = home_avg_goals_scored
df_match_sorted['away_avg_goals_scored'] = away_avg_goals_scored
df_match_sorted['home_avg_goals_conceded'] = home_avg_goals_conceded
df_match_sorted['away_avg_goals_conceded'] = away_avg_goals_conceded

df_match = df_match_sorted
print("Izračun forme (bodovi i golovi) je uspješno završen!")

Pokrećem izračun forme (bodovi i golovi) za zadnjih 5 utakmica...
Izračun forme (bodovi i golovi) je uspješno završen!


### 2.4.5. Glavna tablica

U završnom koraku:
1.  Spajamo sve izračunate značajke u `glavna_df`.
2.  Dodajemo kladioničarske koeficijente (`B365H`, `B365D`, `B365A`) koji služe kao "ekspertna procjena" vjerojatnosti. Njih nećemo koristiti u treniranju, već prilikom evaluacije modela - usporedba našeg modela i modela kladionice.
3.  **Nedostajuće vrijednosti:** Umjesto brisanja redaka, `NaN` vrijednosti popunjavamo prosjekom stupca kako bismo sačuvali veličinu skupa podataka.

In [11]:
betting_stupci = ['B365H', 'B365D', 'B365A', 'BWH', 'BWD', 'BWA']
team_att_stupci = ['home_buildUpSpeed', 'home_defencePressure', 'home_passing', 
                   'away_buildUpSpeed', 'away_defencePressure', 'away_passing']
form_stupci = [
    'home_team_form_points', 'away_team_form_points',
    'home_avg_goals_scored', 'away_avg_goals_scored',
    'home_avg_goals_conceded', 'away_avg_goals_conceded'
]

final_columns = [
    'id', 'date', 'season', 'league_id', 
    'home_team_api_id', 'away_team_api_id', 
    'home_team_rating', 'away_team_rating',
    'result'
] + team_att_stupci + form_stupci + betting_stupci

glavna_df = df_match[final_columns].copy()

team_dict = dict(zip(df_team['team_api_id'], df_team['team_long_name']))
glavna_df['home_team_name'] = glavna_df['home_team_api_id'].map(team_dict)
glavna_df['away_team_name'] = glavna_df['away_team_api_id'].map(team_dict)

numerical_cols = ['home_team_rating', 'away_team_rating'] + team_att_stupci + form_stupci + betting_stupci

for col in numerical_cols:
    glavna_df[col] = glavna_df[col].fillna(glavna_df[col].mean())

new_order = [
    'id', 'date', 'season', 'league_id',
    'home_team_name', 'away_team_name',
    'result',
    'home_team_rating', 'away_team_rating',
    'home_team_form_points', 'away_team_form_points',
    'home_avg_goals_scored', 'away_avg_goals_scored',
    'home_avg_goals_conceded', 'away_avg_goals_conceded',
    'home_buildUpSpeed', 'away_buildUpSpeed',
    'B365H', 'B365D', 'B365A'
]

print(f"Finalna veličina skupa podataka: {glavna_df.shape}")
display(glavna_df[new_order].head(10))

Finalna veličina skupa podataka: (25979, 29)


,id,date,season,league_id,home_team_name,away_team_name,result,home_team_rating,away_team_rating,home_team_form_points,away_team_form_points,home_avg_goals_scored,away_avg_goals_scored,home_avg_goals_conceded,away_avg_goals_conceded,home_buildUpSpeed,away_buildUpSpeed,B365H,B365D,B365A
24558,24559,2008-07-18,2008/2009,24558,BSC Young Boys,FC Basel,2,70.928336,70.856435,0.0,0.0,0.0,0.0,0.0,0.0,52.419813,52.399368,2.628818,3.839684,4.662222
24559,24560,2008-07-19,2008/2009,24558,FC Aarau,FC Sion,1,70.928336,70.856435,0.0,0.0,0.0,0.0,0.0,0.0,52.419813,52.399368,2.628818,3.839684,4.662222
24560,24561,2008-07-20,2008/2009,24558,FC Luzern,FC Vaduz,2,70.928336,70.856435,0.0,0.0,0.0,0.0,0.0,0.0,52.419813,52.399368,2.628818,3.839684,4.662222
24561,24562,2008-07-20,2008/2009,24558,Neuchâtel Xamax,FC Zürich,2,70.928336,70.856435,0.0,0.0,0.0,0.0,0.0,0.0,52.419813,52.399368,2.628818,3.839684,4.662222
24613,24614,2008-07-23,2008/2009,24558,AC Bellinzona,Neuchâtel Xamax,2,70.928336,70.856435,0.0,0.0,0.0,1.0,0.0,2.0,52.419813,52.399368,2.628818,3.839684,4.662222
24612,24613,2008-07-23,2008/2009,24558,FC Basel,Grasshopper Club Zürich,1,70.928336,70.856435,3.0,0.0,2.0,0.0,1.0,0.0,52.419813,52.399368,2.628818,3.839684,4.662222
24614,24615,2008-07-23,2008/2009,24558,FC Zürich,FC Luzern,1,70.928336,70.856435,3.0,0.0,2.0,1.0,1.0,2.0,52.419813,52.399368,2.628818,3.839684,4.662222
24615,24616,2008-07-24,2008/2009,24558,FC Sion,BSC Young Boys,1,70.928336,70.856435,0.0,0.0,1.0,1.0,3.0,2.0,52.419813,52.399368,2.628818,3.839684,4.662222
24616,24617,2008-07-24,2008/2009,24558,FC Vaduz,FC Aarau,2,70.928336,70.856435,3.0,3.0,2.0,3.0,1.0,1.0,52.419813,52.399368,2.628818,3.839684,4.662222
24668,24669,2008-07-26,2008/2009,24558,FC Luzern,Neuchâtel Xamax,2,70.928336,70.856435,0.0,1.5,0.5,1.5,1.5,1.5,52.419813,52.399368,2.628818,3.839684,4.662222


--------------------------------------

# 3. Priprema podataka za model

Skaliranje i slično...

## 4. Implementacija neuronske mreže